# AWS-AlphaFold Load Testing

## 0. Install Dependencies

In [1]:
%pip install -r notebook-requirements.txt -q -q

Note: you may need to restart the kernel to use updated packages.


In [1]:
## Import helper functions at rfutils/rfutils.py
from nbhelpers import nbhelpers, notebook_utils

# Import required Python packages
import boto3
from datetime import datetime
import sagemaker
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
import pandas as pd
import os
from IPython import display
from time import sleep
pd.set_option("max_colwidth", None)

# Get client informatiion
boto_session = boto3.session.Session()
sm_session = sagemaker.session.Session(boto_session)
region = boto_session.region_name
s3_client = boto_session.client("s3", region_name=region)
batch_client = boto_session.client("batch")

S3_BUCKET = sm_session.default_bucket()
print(f" S3 bucket name is {S3_BUCKET}")

 S3 bucket name is sagemaker-us-east-2-032243382548


If you have multiple AWS-Alphafold stacks deployed in your account, which one should we use? If not specified, the `submit_batch_alphafold_job` function defaults to the first item in this list. 

In [2]:
nbhelpers.list_alphafold_stacks()

[{'StackId': 'arn:aws:cloudformation:us-east-2:032243382548:stack/alphafold-22-02-02/627d0e30-8447-11ec-952f-06fa8d1bdf48',
  'StackName': 'alphafold-22-02-02',
  'TemplateDescription': 'Creates a stack for running Alphafold on AWS Batch.',
  'CreationTime': datetime.datetime(2022, 2, 2, 16, 44, 27, 315000, tzinfo=tzutc()),
  'StackStatus': 'CREATE_COMPLETE',
  'DriftInformation': {'StackDriftStatus': 'NOT_CHECKED'}},
 {'StackId': 'arn:aws:cloudformation:us-east-2:032243382548:stack/alphafold-220128/e20cc560-808b-11ec-826e-062a8b73d3aa',
  'StackName': 'alphafold-220128',
  'TemplateDescription': 'Creates a stack for running Alphafold on AWS Batch.',
  'CreationTime': datetime.datetime(2022, 1, 28, 22, 44, 42, 468000, tzinfo=tzutc()),
  'LastUpdatedTime': datetime.datetime(2022, 1, 31, 18, 5, 29, 981000, tzinfo=tzutc()),
  'StackStatus': 'UPDATE_COMPLETE',
  'DriftInformation': {'StackDriftStatus': 'NOT_CHECKED'}}]

In [7]:
job_name_list = []

id_1 = "T1028"
sequence_1 = "MARIGDLDAARPAPEAVPGDMVRIPGGTFLQGSPERTLDWLDREGQAFPRDWFTDETPQIPVTLPDYLIDRHQVTVAQFAAFVSRTGYVTSAERAGGSMVYGEQYWEIREGACWHRPAGYGSGIRGRDDHPVVHISFADAEAYARWAGRRLPTESEWERAATGPSYRLWPWGDTWDSRNANTAEHTAGALGDLDAWRTWWGAIHAVQGPMPQTTPVGAFSPRGDSVDGCADMTGNVYEWTSTLAHLYSPATRCDPTIHLVMGRSRVIRGGSWMNFRYQVRCAERLYGDPTGWSNFALGFRCARDVTAVPHVDDNGR"

input_ids = (id_1,)
input_sequences = (sequence_1,)

# If folding a complex target and all the input sequences are
# prokaryotic then set `is_prokaryotic` to `True`. Set to `False`
# otherwise or if the origin is unknown.
IS_PROKARYOTE = False
MIN_SINGLE_SEQUENCE_LENGTH = 16
MAX_SINGLE_SEQUENCE_LENGTH = 2500
MAX_MULTIMER_LENGTH = 2500
MAX_TEMPLATE_DATE = "2022-01-01"
DB_PRESET = "full_dbs"
RUN_FEATURES_ONLY = False
BENCHMARK = False
USE_PRECOMPUTED_MSAS = False

sequences, model_type_to_use = notebook_utils.validate_input(
    input_sequences=input_sequences,
    min_length=MIN_SINGLE_SEQUENCE_LENGTH,
    max_length=MAX_SINGLE_SEQUENCE_LENGTH,
    max_multimer_length=MAX_MULTIMER_LENGTH,
)

model_preset = "monomer"
model_names = notebook_utils.MODEL_PRESETS["monomer"] + ("model_2_ptm",)

vCPU_options = [4, 8, 16, 32]
MEM_options = [8, 16, 32]

# vCPU_options = [8]
# MEM_options = [16]
sequence_length = len(sequence_1)

for cpu in vCPU_options:
    for mem in MEM_options:
        # job_name = nbhelpers.create_job_name()
        job_name = f"20220207-{sequence_length}-{id_1}-{cpu}-{mem}-{1}"
        object_key = nbhelpers.upload_fasta_to_s3(
            sequences, input_ids, S3_BUCKET, job_name, region=region
        )

        sequences, model_type_to_use = notebook_utils.validate_input(
            input_sequences=input_sequences,
            min_length=MIN_SINGLE_SEQUENCE_LENGTH,
            max_length=MAX_SINGLE_SEQUENCE_LENGTH,
            max_multimer_length=MAX_MULTIMER_LENGTH,
        )

        model_preset = "monomer"
        model_names = notebook_utils.MODEL_PRESETS["monomer"] + ("model_2_ptm",)

        # Upload input file to S3
        object_key = nbhelpers.upload_fasta_to_s3(
            sequences, input_ids, S3_BUCKET, job_name, region=region
        )

        # Define resources for data prep and prediction steps
        prep_cpu = cpu
        prep_mem = mem
        predict_cpu = cpu
        predict_mem = mem
        predict_gpu = 1

        step_1_response = nbhelpers.submit_batch_alphafold_job(
            job_name=str(job_name),
            fasta_paths=object_key,
            output_dir=job_name,
            db_preset=DB_PRESET,
            model_preset=model_preset,
            s3_bucket=S3_BUCKET,
            cpu=prep_cpu,
            memory=prep_mem,
            gpu=0,
            run_features_only=True,
        )

        print(f"Job ID {step_1_response['jobId']} submitted")

        step_2_response = nbhelpers.submit_batch_alphafold_job(
            job_name=str(job_name),
            fasta_paths=object_key,
            output_dir=job_name,
            db_preset=DB_PRESET,
            model_preset=model_preset,
            s3_bucket=S3_BUCKET,
            cpu=predict_cpu,
            memory=predict_mem,
            gpu=predict_gpu,
            features_paths=os.path.join(job_name, "features.pkl"),
            depends_on=step_1_response["jobId"],
        )

        print(f"Job ID {step_2_response['jobId']} submitted")
        job_info = [job_name, step_2_response['jobId'], id_1, sequence_length]
        job_name_list.append(job_info)
        sleep(0.5)
df = pd.DataFrame(job_name_list, columns = ['JobName', 'JobId', 'SequenceId', "SequenceLength"])
df

Using the single-chain model.
Sequence file uploaded to s3://sagemaker-us-east-2-032243382548/20220207-316-T1028-4-8-1/20220207-316-T1028-4-8-1.fasta
Using the single-chain model.
Sequence file uploaded to s3://sagemaker-us-east-2-032243382548/20220207-316-T1028-4-8-1/20220207-316-T1028-4-8-1.fasta
{'command': ['--fasta_paths=20220207-316-T1028-4-8-1/20220207-316-T1028-4-8-1.fasta', '--uniref90_database_path=/mnt/uniref90_database_path/uniref90.fasta', '--mgnify_database_path=/mnt/mgnify_database_path/mgy_clusters_2018_12.fa', '--data_dir=/mnt/data_dir/fsx', '--template_mmcif_dir=/mnt/template_mmcif_dir/mmcif_files', '--obsolete_pdbs_path=/mnt/obsolete_pdbs_path/obsolete.dat', '--output_dir=20220207-316-T1028-4-8-1', '--max_template_date=2022-02-07', '--db_preset=full_dbs', '--model_preset=monomer', '--s3_bucket=sagemaker-us-east-2-032243382548', '--pdb70_database_path=/mnt/pdb70_database_path/pdb70', '--uniclust30_database_path=/mnt/uniclust30_database_path/uniclust30_2018_08/uniclust

,JobName,JobId,SequenceId,SequenceLength
0,20220207-316-T1028-4-8-1,a12fe9e0-dcfb-42fe-9424-917ee482c87c,T1028,316
1,20220207-316-T1028-4-16-1,9092e457-741b-4530-bb99-2383ef79498c,T1028,316
2,20220207-316-T1028-4-32-1,e608d807-8b26-492d-ba35-1eab4b1cbcb3,T1028,316
3,20220207-316-T1028-8-8-1,439420db-36f5-4d97-8f7d-c81ab9a39924,T1028,316
4,20220207-316-T1028-8-16-1,cb669ea7-51b1-44aa-99ca-e0db373beb6b,T1028,316
5,20220207-316-T1028-8-32-1,e75f31e0-00de-4ca5-81f8-594efda1a44d,T1028,316
6,20220207-316-T1028-16-8-1,81eb53d2-b88e-4130-9913-e8b2c07e9991,T1028,316
7,20220207-316-T1028-16-16-1,08b1cf99-78f0-49f3-9f0e-89932a1def0c,T1028,316
8,20220207-316-T1028-16-32-1,78288451-f21b-4bb0-9b27-c8422056f04d,T1028,316
9,20220207-316-T1028-32-8-1,64de0817-d933-441d-bb7d-532834325565,T1028,316


In [4]:
nbhelpers.get_run_metrics(S3_BUCKET, "20220207-316-T1028-16-32-1")

(                             duration_sec
 process_features_model_1         3.918499
 predict_and_compile_model_1    257.876479
 relax_model_1                   48.777422
 process_features_model_2         2.814968
 predict_and_compile_model_2    244.521688
 relax_model_2                   39.445560
 process_features_model_3         2.516922
 predict_and_compile_model_3    225.076089
 relax_model_3                   40.184604
 process_features_model_4         2.549720
 predict_and_compile_model_4    221.884838
 relax_model_4                   39.403420
 process_features_model_5         2.592879
 predict_and_compile_model_5    205.930521
 relax_model_5                   40.484865,
             plddts
 model_1  91.632372
 model_2  92.556485
 model_3  91.257135
 model_4  91.709964
 model_5  91.520733,
          0
 0  model_2
 1  model_4
 2  model_1
 3  model_5
 4  model_3)

---

Download and process CASP14 sequences

In [7]:
!wget "https://predictioncenter.org/download_area/CASP14/sequences/casp14.seq.txt" -O "data/casp14.fa"

--2022-02-07 16:01:56--  https://predictioncenter.org/download_area/CASP14/sequences/casp14.seq.txt
Resolving predictioncenter.org (predictioncenter.org)... 128.120.136.155
Connecting to predictioncenter.org (predictioncenter.org)|128.120.136.155|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 31420 (31K) [text/plain]
Saving to: ‘data/casp14.fa’

data/casp14.fa      100%[===================>]  30.68K  --.-KB/s    in 0.1s    

2022-02-07 16:01:57 (233 KB/s) - ‘data/casp14.fa’ saved [31420/31420]



In [8]:
!sed '137,138d' "data/casp14.fa" > "data/casp14_dedup.fa" # Remove duplicate entry for T1085

In [9]:
casp14_iterator = SeqIO.parse("data/casp14_dedup.fa", "fasta")
casp14_df = pd.DataFrame(
    (
        (record.id, record.description, len(record), record.seq)
        for record in casp14_iterator
    ),
    columns=["id", "description", "length", "seq"],
).sort_values(by="length")
!rm data/casp14*

Display information about CASP14 proteins

Plot distribution of the protein lengths

Submit analysis jobs for a subset of CASP14 proteins

In [12]:
protein_count = (
    85  # Change this to analyze a larger number of CASP14 targets, smallest to largest
)
job_name_list = []

# If folding a complex target and all the input sequences are
# prokaryotic then set `is_prokaryotic` to `True`. Set to `False`
# otherwise or if the origin is unknown.
IS_PROKARYOTE = False

MIN_SINGLE_SEQUENCE_LENGTH = 16
MAX_SINGLE_SEQUENCE_LENGTH = 2500
MAX_MULTIMER_LENGTH = 2500
MAX_TEMPLATE_DATE = "2022-01-01"
DB_PRESET = "full_dbs"
RUN_FEATURES_ONLY = False
BENCHMARK = False
USE_PRECOMPUTED_MSAS = False

for row in casp14_df[:protein_count].itertuples(index=False):
    record = SeqRecord(row.seq, id=row.id, description=row.description)
    print(f"Protein sequence for analysis is \n{record.description}")
    sequence_length = len(record.seq)
    print(f"Sequence length is {sequence_length}")
    print(record.seq)

    input_ids = (record.id,)
    input_sequences = (str(record.seq),)

    sequences, model_type_to_use = notebook_utils.validate_input(
        input_sequences=input_sequences,
        min_length=MIN_SINGLE_SEQUENCE_LENGTH,
        max_length=MAX_SINGLE_SEQUENCE_LENGTH,
        max_multimer_length=MAX_MULTIMER_LENGTH,
    )

    if model_type_to_use == notebook_utils.ModelType.MONOMER:
        model_preset = "monomer"
        model_names = notebook_utils.MODEL_PRESETS["monomer"] + ("model_2_ptm",)
    elif model_type_to_use == notebook_utils.ModelType.MULTIMER:
        model_preset = "multimer"
        model_names = notebook_utils.MODEL_PRESETS["multimer"]

    # Upload input file to S3
    # job_name = nbhelpers.create_job_name()
    job_name = f"20220207-{sequence_length}-{record.id}-8-32-4-16-1"

    object_key = nbhelpers.upload_fasta_to_s3(
        sequences, input_ids, S3_BUCKET, job_name, region=region
    )

    # Define resources for data prep and prediction steps
    # if sequence_length < 400:
    #     prep_cpu = 8
    #     prep_mem = 32
    #     predict_cpu = 4
    #     predict_mem = 16
    #     predict_gpu = 1
    # else:
    #     prep_cpu = 8
    #     prep_mem = 64
    #     predict_cpu = 4
    #     predict_mem = 32
    #     predict_gpu = 0

    prep_cpu = 8
    prep_mem = 32
    predict_cpu = 4
    predict_mem = 16
    predict_gpu = 1

    step_1_response = nbhelpers.submit_batch_alphafold_job(
        job_name=str(job_name),
        fasta_paths=object_key,
        output_dir=job_name,
        db_preset=DB_PRESET,
        model_preset=model_preset,
        s3_bucket=S3_BUCKET,
        cpu=prep_cpu,
        memory=prep_mem,
        gpu=0,
        run_features_only=True,
    )

    print(f"Job ID {step_1_response['jobId']} submitted")

    step_2_response = nbhelpers.submit_batch_alphafold_job(
        job_name=str(job_name),
        fasta_paths=object_key,
        output_dir=job_name,
        db_preset=DB_PRESET,
        model_preset=model_preset,
        s3_bucket=S3_BUCKET,
        cpu=predict_cpu,
        memory=predict_mem,
        gpu=predict_gpu,
        features_paths=os.path.join(job_name, "features.pkl"),
        depends_on=step_1_response["jobId"],
    )

    job_info = [job_name, step_2_response['jobId'], record.id, sequence_length]
    job_name_list.append(job_info)
    sleep(0.5)
df = pd.DataFrame(job_name_list, columns = ['JobName', 'JobId', 'SequenceId', "SequenceLength"])
df

Protein sequence for analysis is 
T1059 MtrunA17_Chr7g0216231, Medicago truncatula, 32 residues|
Sequence length is 32
TKPCQSDKDCKKFACRKPKVPKCINGFCKCVR
Using the single-chain model.
Sequence file uploaded to s3://sagemaker-us-east-2-032243382548/20220207-32-T1059-8-32-4-16-1/20220207-32-T1059-8-32-4-16-1.fasta
{'command': ['--fasta_paths=20220207-32-T1059-8-32-4-16-1/20220207-32-T1059-8-32-4-16-1.fasta', '--uniref90_database_path=/mnt/uniref90_database_path/uniref90.fasta', '--mgnify_database_path=/mnt/mgnify_database_path/mgy_clusters_2018_12.fa', '--data_dir=/mnt/data_dir/fsx', '--template_mmcif_dir=/mnt/template_mmcif_dir/mmcif_files', '--obsolete_pdbs_path=/mnt/obsolete_pdbs_path/obsolete.dat', '--output_dir=20220207-32-T1059-8-32-4-16-1', '--max_template_date=2022-02-07', '--db_preset=full_dbs', '--model_preset=monomer', '--s3_bucket=sagemaker-us-east-2-032243382548', '--pdb70_database_path=/mnt/pdb70_database_path/pdb70', '--uniclust30_database_path=/mnt/uniclust30_database_path/

,JobName,JobId,SequenceId,SequenceLength
0,20220207-32-T1059-8-32-4-16-1,e0999162-1bbf-4da9-a173-6fc74b86c95f,T1059,32
1,20220207-35-T1062-8-32-4-16-1,2735b6fe-3744-4c4e-a694-f9236161d3d8,T1062,35
2,20220207-69-T1072s2-8-32-4-16-1,c39c8502-b70b-4300-8a8e-bfec7c91e2af,T1072s2,69
3,20220207-73-T1084-8-32-4-16-1,471644bf-3303-4cc5-bbbf-937d448dc768,T1084,73
4,20220207-74-T1046s1-8-32-4-16-1,f2e132cd-e469-4f01-98ef-67fdc5da27e5,T1046s1,74
...,...,...,...,...
79,20220207-832-T1052-8-32-4-16-1,cdc6f06f-618e-4c64-829d-9c4c9fc29936,T1052,832
80,20220207-863-T1091-8-32-4-16-1,05d80834-9682-4c69-bfe8-62502b71943f,T1091,863
81,20220207-922-T1080-8-32-4-16-1,3362a8c9-2410-47f2-bd83-3583b5770c47,T1080,922
82,20220207-949-T1061-8-32-4-16-1,7f5ba0e6-3ee1-494d-8c6e-44feda6a95c7,T1061,949


In [13]:
df.to_csv("220207_PM_runs.csv")

---
2/8/2022
Several OOM failures seen last night with 8-32-4-16-1:
Data Prep:
- T1050
- T1098
- T1071
- T1024
- T1086
- T1025

Predict:
- T1044

Retry with 8-64-4-16-2

In [14]:
protein_count = (
    85  # Change this to analyze a larger number of CASP14 targets, smallest to largest
)
job_name_list = []

# If folding a complex target and all the input sequences are
# prokaryotic then set `is_prokaryotic` to `True`. Set to `False`
# otherwise or if the origin is unknown.
IS_PROKARYOTE = False

MIN_SINGLE_SEQUENCE_LENGTH = 16
MAX_SINGLE_SEQUENCE_LENGTH = 2500
MAX_MULTIMER_LENGTH = 2500
MAX_TEMPLATE_DATE = "2022-01-01"
DB_PRESET = "full_dbs"
RUN_FEATURES_ONLY = False
BENCHMARK = False
USE_PRECOMPUTED_MSAS = False

for row in casp14_df[:protein_count].itertuples(index=False):
    record = SeqRecord(row.seq, id=row.id, description=row.description)
    print(f"Protein sequence for analysis is \n{record.description}")
    sequence_length = len(record.seq)
    print(f"Sequence length is {sequence_length}")
    print(record.seq)

    input_ids = (record.id,)
    input_sequences = (str(record.seq),)

    sequences, model_type_to_use = notebook_utils.validate_input(
        input_sequences=input_sequences,
        min_length=MIN_SINGLE_SEQUENCE_LENGTH,
        max_length=MAX_SINGLE_SEQUENCE_LENGTH,
        max_multimer_length=MAX_MULTIMER_LENGTH,
    )

    if model_type_to_use == notebook_utils.ModelType.MONOMER:
        model_preset = "monomer"
        model_names = notebook_utils.MODEL_PRESETS["monomer"] + ("model_2_ptm",)
    elif model_type_to_use == notebook_utils.ModelType.MULTIMER:
        model_preset = "multimer"
        model_names = notebook_utils.MODEL_PRESETS["multimer"]

    # Upload input file to S3
    # job_name = nbhelpers.create_job_name()
    job_name = f"20220207-{sequence_length}-{record.id}-8-32-4-16-1"

    object_key = nbhelpers.upload_fasta_to_s3(
        sequences, input_ids, S3_BUCKET, job_name, region=region
    )

    # Define resources for data prep and prediction steps
    # if sequence_length < 400:
    #     prep_cpu = 8
    #     prep_mem = 32
    #     predict_cpu = 4
    #     predict_mem = 16
    #     predict_gpu = 1
    # else:
    #     prep_cpu = 8
    #     prep_mem = 64
    #     predict_cpu = 4
    #     predict_mem = 32
    #     predict_gpu = 0

    prep_cpu = 8
    prep_mem = 64
    predict_cpu = 4
    predict_mem = 16
    predict_gpu = 2

    step_1_response = nbhelpers.submit_batch_alphafold_job(
        job_name=str(job_name),
        fasta_paths=object_key,
        output_dir=job_name,
        db_preset=DB_PRESET,
        model_preset=model_preset,
        s3_bucket=S3_BUCKET,
        cpu=prep_cpu,
        memory=prep_mem,
        gpu=0,
        run_features_only=True,
    )

    print(f"Job ID {step_1_response['jobId']} submitted")

    step_2_response = nbhelpers.submit_batch_alphafold_job(
        job_name=str(job_name),
        fasta_paths=object_key,
        output_dir=job_name,
        db_preset=DB_PRESET,
        model_preset=model_preset,
        s3_bucket=S3_BUCKET,
        cpu=predict_cpu,
        memory=predict_mem,
        gpu=predict_gpu,
        features_paths=os.path.join(job_name, "features.pkl"),
        depends_on=step_1_response["jobId"],
    )

    job_info = [job_name, step_2_response['jobId'], record.id, sequence_length]
    job_name_list.append(job_info)
    sleep(0.5)
df = pd.DataFrame(job_name_list, columns = ['JobName', 'JobId', 'SequenceId', "SequenceLength"])
df.to_csv("220208_AM_runs.csv")

Protein sequence for analysis is 
T1059 MtrunA17_Chr7g0216231, Medicago truncatula, 32 residues|
Sequence length is 32
TKPCQSDKDCKKFACRKPKVPKCINGFCKCVR
Using the single-chain model.
Sequence file uploaded to s3://sagemaker-us-east-2-032243382548/20220207-32-T1059-8-32-4-16-1/20220207-32-T1059-8-32-4-16-1.fasta
{'command': ['--fasta_paths=20220207-32-T1059-8-32-4-16-1/20220207-32-T1059-8-32-4-16-1.fasta', '--uniref90_database_path=/mnt/uniref90_database_path/uniref90.fasta', '--mgnify_database_path=/mnt/mgnify_database_path/mgy_clusters_2018_12.fa', '--data_dir=/mnt/data_dir/fsx', '--template_mmcif_dir=/mnt/template_mmcif_dir/mmcif_files', '--obsolete_pdbs_path=/mnt/obsolete_pdbs_path/obsolete.dat', '--output_dir=20220207-32-T1059-8-32-4-16-1', '--max_template_date=2022-02-07', '--db_preset=full_dbs', '--model_preset=monomer', '--s3_bucket=sagemaker-us-east-2-032243382548', '--pdb70_database_path=/mnt/pdb70_database_path/pdb70', '--uniclust30_database_path=/mnt/uniclust30_database_path/

### Interpreting the prediction

In general predicted LDDT (pLDDT) is best used for intra-domain confidence, whereas Predicted Aligned Error (PAE) is best used for determining between domain or between chain confidence.

Please see the [AlphaFold methods paper](https://www.nature.com/articles/s41586-021-03819-2), the [AlphaFold predictions of the human proteome paper](https://www.nature.com/articles/s41586-021-03828-1), and the [AlphaFold-Multimer paper](https://www.biorxiv.org/content/10.1101/2021.10.04.463034v1) as well as [our FAQ](https://alphafold.ebi.ac.uk/faq) on how to interpret AlphaFold predictions.